# General setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sksurv.metrics import integrated_brier_score, cumulative_dynamic_auc, concordance_index_ipcw

from statsmodels.miscmodels.ordinal_model import OrderedModel

In [2]:
# read the data
test = pd.read_csv("test_fin.csv", index_col=0).fillna(0)
train = pd.read_csv("train_fin.csv", index_col=0).fillna(0)
val = pd.read_csv("val_fin.csv", index_col=0).fillna(0)

# Define the dictionary with the mapping
mapping = {1: False, 2: True}

# Apply the mapping to the 'col1' column
train['status'] = train['status'].replace(mapping)
test['status'] = test['status'].replace(mapping)
val['status'] = val['status'].replace(mapping)

y_train = np.array(
    list(zip(train['status'], train['time_to_event'])),
    dtype=[('status', '?'), ('time_to_event', 'double')]
)

y_test = np.array(
    list(zip(test['status'], test['time_to_event'])),
    dtype=[('status', '?'), ('time_to_event', 'double')]
)

times_data = np.arange(1, 70).tolist()
train.shape, test.shape, y_train.shape, y_test.shape

((12613, 55), (5409, 55), (12613,), (5409,))

In [3]:
train['time_to_event_CAT'] = np.where(train['status']== True, train['time_to_event'], -train['time_to_event'])
test['time_to_event_CAT'] = np.where(test['status']== True, test['time_to_event'], -test['time_to_event'])
val['time_to_event_CAT'] = np.where(val['status']== True, val['time_to_event'], -val['time_to_event'])

In [4]:
# evaluation metrics definition 
    
def calc_metrics(y_train, y_test, y_pred, times):

    metrics = {
        "cumulative_dynamic_auc": cumulative_dynamic_auc(y_train, y_test, y_pred, times),
        "concordance_index_ipcw": concordance_index_ipcw(y_train, y_test, y_pred)
    }
    return metrics


def display_metric(metrics):
    print(metrics["cumulative_dynamic_auc"])
    print(metrics["concordance_index_ipcw"])

#y_train = np.array([(1, 3), (0, 9) , (1, 0), (1, 8), (0, 9), (1,1)])
#y_test = np.array([(0, 3), (1, 9) , (0, 0), (1, 8), (0, 9), (1,1)])
#y_pred = np.array([(1, 3), (0, 2) , (1, 1), (1, 8), (0, 9), (1,1)])

#print(calc_metrics(y_train, y_test, y_pred, times)["classification_report"])

# Random Forest

In [75]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']

In [7]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV
import time

start = time.time()
estimator = RandomSurvivalForest()

parameter_space = {
     'max_depth': [2, 3, 4, 5, 8],
     'max_features': ['log2', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [10, 25, 50, 75, 100, 200]
}

cross_validation_schema = ShuffleSplit(n_splits=3)

n_iterations = 1

random_search_cv_1 = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=parameter_space,
    n_iter=n_iterations,
    n_jobs=-1,
    cv=cross_validation_schema,
    random_state=123,
    return_train_score=False,
    verbose=3 #Controls the verbosity: the higher, the more messages.

)
rf_search_1 = random_search_cv_1.fit(train[features], y_train)

end = time.time()
print("elapsed: ", end - start)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
elapsed:  14.032626867294312


In [83]:
def RF_survival(train, test, features, y_train, params):
    clf = RandomSurvivalForest(**params)
    clf.fit(train[features], y_train)
    preds = clf.predict(test[features])
    return preds, clf

def RF_survival_result_wrapper(train, test, features, y_train, y_test, times, verbose, params):
    model_output, model = RF_survival(
        train, test, features, y_train, params
    )
    results = calc_metrics(y_train, y_test, model_output, times)
    return results, model_output, model

In [84]:
results_RF, output, RF_model = RF_survival_result_wrapper(
    train,
    test,
    features,
    y_train,
    y_test,
    times_data,
    False,
    rf_search_1.best_params_
)

print(results_RF)
print(output)

{'cumulative_dynamic_auc': (array([0.9640639 , 0.93039611, 0.93334606, 0.92611015, 0.92633715,
       0.93003311, 0.93269675, 0.93527036, 0.93419775, 0.93476598,
       0.93645336, 0.93725814, 0.9375644 , 0.93815811, 0.93811723,
       0.93966345, 0.94153021, 0.9410524 , 0.94354454, 0.94157556,
       0.93913982, 0.93999631, 0.94088523, 0.94304688, 0.9460297 ,
       0.9483384 , 0.95003933, 0.95131511, 0.95216283, 0.95325806,
       0.95400699, 0.95579434, 0.9578193 , 0.9568102 , 0.95776264,
       0.95949324, 0.9602784 , 0.96208382, 0.96331747, 0.9655946 ,
       0.96780712, 0.96567501, 0.9653051 , 0.96773053, 0.96811923,
       0.96790248, 0.96875167, 0.96843004, 0.969144  , 0.95031173,
       0.94298763, 0.94148834, 0.94345655, 0.94165075, 0.94274767,
       0.94223145, 0.94303265, 0.94382248, 0.94395173, 0.9436104 ,
       0.94552136, 0.94221778, 0.94185208, 0.94343782, 0.9448899 ,
       0.94496821, 0.94842136, 0.94954129, 0.95105384]), 0.947419407318438), 'concordance_index_ipcw'

In [79]:
rf_search_1.best_params_

{'n_estimators': 10,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 8}

[CV 2/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.896 total time=   3.7s
[CV 1/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.904 total time=   5.1s
[CV 3/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.924 total time=   5.8s


In [85]:
shap_values = shap.TreeExplainer(RF_model).shap_values(test[features])

InvalidModelError: Model type not yet supported by TreeExplainer: <class 'sksurv.ensemble.forest.RandomSurvivalForest'>

# Gradient Boosted Cox model

In [12]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']

In [13]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis

start = time.time()
estimator = GradientBoostingSurvivalAnalysis()

parameter_space = {
     'max_depth': [2, 3, 4, 5, 8],
     'max_features': ['log2', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [10, 25, 50, 75, 100, 200],
     'learning_rate': [0.1, 0.01, 0.005, 0.001]
}

cross_validation_schema = ShuffleSplit(n_splits=3)

n_iterations = 30

random_search_cv_1 = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=parameter_space,
    n_iter=n_iterations,
    n_jobs=-1,
    cv=cross_validation_schema,
    random_state=123,
    return_train_score=False,
    verbose=3 #Controls the verbosity: the higher, the more messages.

)
GBC_search_1 = random_search_cv_1.fit(train[features], y_train)

end = time.time()
print("elapsed: ", end - start)

print(GBC_search_1.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
elapsed:  2231.154829978943
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 8, 'learning_rate': 0.1}


In [14]:
def GBC_survival(train, test, features, y_train, params):
    clf = GradientBoostingSurvivalAnalysis(**params)
    clf.fit(train[features], y_train)
    preds = clf.predict(test[features])
    return preds, clf

def GBC_survival_result_wrapper(train, test, features, y_train, y_test, times, verbose, params):
    model_output, model = GBC_survival(
        train, test, features, y_train, params
    )
    results = calc_metrics(y_train, y_test, model_output, times)
    return results, model

In [18]:
results_GBC, model = GBC_survival_result_wrapper(
    train,
    test,
    features,
    y_train,
    y_test,
    times_data,
    False,
    GBC_search_1.best_params_
)

print(results_GBC)

{'cumulative_dynamic_auc': (array([0.97623493, 0.95796109, 0.96067957, 0.95349818, 0.95278489,
       0.95624002, 0.95843844, 0.9609077 , 0.9602552 , 0.96038156,
       0.9610007 , 0.96210104, 0.96222434, 0.96239795, 0.96232456,
       0.96362271, 0.96506972, 0.96459935, 0.96597508, 0.96252426,
       0.95965822, 0.96015873, 0.9610724 , 0.9630192 , 0.96528994,
       0.96737988, 0.96805581, 0.96889492, 0.9689586 , 0.96973764,
       0.97080505, 0.97209988, 0.97365071, 0.97282275, 0.97301358,
       0.97416397, 0.974929  , 0.9763289 , 0.97724717, 0.97854026,
       0.97959263, 0.97665705, 0.97637102, 0.97801468, 0.97795184,
       0.97784057, 0.97832496, 0.97822002, 0.97864464, 0.96204088,
       0.95488753, 0.95218367, 0.95348005, 0.9505172 , 0.95184253,
       0.9514788 , 0.95227654, 0.95301781, 0.95245862, 0.95178278,
       0.95354485, 0.95160006, 0.94989032, 0.9510048 , 0.9518676 ,
       0.95263401, 0.95491149, 0.95602238, 0.95713518]), 0.9650253653619675), 'concordance_index_ipcw

# SVM

In [19]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_standardized = pd.DataFrame(scaler.fit_transform(train[features]))
test_standardized = pd.DataFrame(scaler.transform(test[features]))

train_standardized.columns = features
test_standardized.columns = features

In [ ]:
from sksurv.svm import FastKernelSurvivalSVM
import time
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV
start = time.time()
estimator = FastKernelSurvivalSVM()

parameter_space = {
     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
     'alpha': [0.1, 0.5, 1],
     'gamma': [None, 0.1, 0.5, 1]
}

cross_validation_schema = ShuffleSplit(n_splits=3)

n_iterations = 30

random_search_cv_1 = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=parameter_space,
    n_iter=n_iterations,
    n_jobs=-1,
    cv=cross_validation_schema,
    random_state=123,
    return_train_score=False,
    verbose=3 #Controls the verbosity: the higher, the more messages.

)
SVM_search_1 = random_search_cv_1.fit(train_standardized[features], y_train)

end = time.time()
print("elapsed: ", end - start)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


In [ ]:
def SVM_survival(train, test, features, y_train, params):
    clf = FastKernelSurvivalSVM(**params)
    clf.fit(train[features], y_train)
    preds = clf.predict(test[features])
    return preds

def SVM_survival_result_wrapper(train, test, features, y_train, y_test, times, verbose, params):
    model_output = SVM_survival(
        train, test, features, y_train, params
    )
    results = calc_metrics(y_train, y_test, model_output, times)
    return results

In [207]:
results_SVM = SVM_survival_result_wrapper(
    train_standardized,
    test_standardized,
    features,
    y_train,
    y_test,
    times_data,
    False,
    SVM_search_1.best_params_
)

print(results_SVM)

/var/folders/dc/k7crvlf16t786rmzsdp6xqs00000gn/T/ipykernel_62589/3417522465.py:3: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  clf.fit(train[features], y_train)


{'cumulative_dynamic_auc': (array([0.97585663, 0.96238065, 0.96441039, 0.9610304 , 0.96057529,
       0.96323312, 0.96437301, 0.9672231 , 0.96822594, 0.9686265 ,
       0.96813308, 0.9689092 , 0.96924969, 0.97024592, 0.9706812 ,
       0.97195384, 0.97332665, 0.97327144, 0.97410493, 0.97238714,
       0.97047228, 0.97135892, 0.9721961 , 0.97332455, 0.97490007,
       0.97597246, 0.97594646, 0.97723588, 0.97838273, 0.97849575,
       0.9782466 , 0.97893111, 0.97996582, 0.98019293, 0.98096562,
       0.98211741, 0.98246959, 0.98308681, 0.9841747 , 0.98518385,
       0.98633482, 0.98578229, 0.98502034, 0.98639046, 0.9863326 ,
       0.98583082, 0.98661272, 0.98615624, 0.98394963, 0.9723921 ,
       0.96746869, 0.96505313, 0.96556142, 0.96244674, 0.96332916,
       0.96313919, 0.96384784, 0.96363388, 0.96239836, 0.96230222,
       0.96301079, 0.95975294, 0.96011914, 0.96113848, 0.96178202,
       0.9616722 , 0.9631701 , 0.9636585 , 0.96390657]), 0.9725057189797467), 'concordance_index_ipcw

/opt/homebrew/Caskroom/miniforge/base/envs/virt/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)


[CV 1/3] END .alpha=0.5, gamma=None, kernel=rbf;, score=0.953 total time= 1.6min


/opt/homebrew/Caskroom/miniforge/base/envs/virt/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)


[CV 1/3] END ............alpha=0.5, kernel=poly;, score=0.892 total time=  10.7s
[CV 3/3] END .alpha=0.5, gamma=None, kernel=rbf;, score=0.940 total time= 1.7min


/opt/homebrew/Caskroom/miniforge/base/envs/virt/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: ConvergenceWarning: Optimization did not converge: Warning: Maximum number of iterations has been exceeded.
  estimator.fit(X_train, y_train, **fit_params)


# CoxPH - uważać czy full MLE czy partial MLE

In [322]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']

In [78]:
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis

def CoxPH(train, test, features, y_train):
    clf = CoxnetSurvivalAnalysis(alpha_min_ratio=0.0000000001)
    clf.fit(train[features], y_train)
    preds = clf.predict(test[features])
    return preds

def CoxPH_result_wrapper(train, test, features, y_train, y_test, times, verbose):
    model_output = CoxPH(
        train, test, features, y_train
    )
    print(model_output)
    results = calc_metrics(y_train, y_test, model_output, times)
    return results

In [79]:
results_CoxPH = CoxPH_result_wrapper(
    train,
    test,
    features,
    y_train,
    y_test,
    times_data,
    False
)

print(results_CoxPH)

[ 3.23680198 -1.89893868 -2.25677145 ...  1.75710022 -2.10793946
 -0.99585944]
{'cumulative_dynamic_auc': (array([0.95591541, 0.93050849, 0.93098464, 0.92680874, 0.92707979,
       0.92661775, 0.92882035, 0.92833246, 0.92877831, 0.92843606,
       0.92650988, 0.92725441, 0.92624777, 0.92796277, 0.92615412,
       0.92742071, 0.92851101, 0.92845724, 0.9292104 , 0.92699317,
       0.92481228, 0.92705387, 0.92871773, 0.93136306, 0.93441351,
       0.93679101, 0.93781157, 0.93955903, 0.94068027, 0.94165893,
       0.94241289, 0.94455029, 0.94589173, 0.9458088 , 0.94755544,
       0.94991099, 0.94989514, 0.9505056 , 0.95228266, 0.95491225,
       0.95664737, 0.95465448, 0.95380289, 0.95569853, 0.9557101 ,
       0.9555268 , 0.95725701, 0.95699039, 0.95579799, 0.93601016,
       0.92709703, 0.92592783, 0.9271408 , 0.92490187, 0.92645675,
       0.92639176, 0.9289568 , 0.93008427, 0.92935168, 0.92914896,
       0.93126967, 0.92817348, 0.93033302, 0.93343349, 0.93463303,
       0.93578978, 0.9

# CoxPH Elastic Net

In [5]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']

In [6]:
len(features)

51

In [7]:
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis

def CoxPH_elastic_net(train, test, features, y_train):
    clf = CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.01)
    clf.fit(train[features], y_train)
    preds = clf.predict(test[features])
    return preds

def CoxPH_elastic_net_result_wrapper(train, test, features, y_train, y_test, times, verbose):
    model_output = CoxPH_elastic_net(
        train, test, features, y_train
    )
    results = calc_metrics(y_train, y_test, model_output, times)
    return results

In [8]:
results_CoxPH_elastic_net = CoxPH_elastic_net_result_wrapper(
    train,
    test,
    features,
    y_train,
    y_test,
    times_data,
    False
)

print(results_CoxPH_elastic_net)

{'cumulative_dynamic_auc': (array([0.79289933, 0.79804003, 0.79498516, 0.78516331, 0.7845892 ,
       0.79300953, 0.79410012, 0.80252137, 0.80434738, 0.80846009,
       0.80733134, 0.81435846, 0.8193588 , 0.82281734, 0.82386557,
       0.82685245, 0.83087044, 0.83272789, 0.83680701, 0.8366192 ,
       0.83675338, 0.8409931 , 0.84439392, 0.84743422, 0.85185212,
       0.85506367, 0.85613787, 0.8592648 , 0.86092361, 0.86198819,
       0.86167847, 0.86200542, 0.85980335, 0.86174869, 0.86370821,
       0.86571611, 0.86921171, 0.86975172, 0.87059128, 0.87393784,
       0.87336672, 0.87419702, 0.87059094, 0.87220332, 0.8718722 ,
       0.86966343, 0.86699187, 0.86745832, 0.86732695, 0.84005605,
       0.83269275, 0.83300132, 0.83307961, 0.8314125 , 0.83404954,
       0.83488646, 0.83625049, 0.83648748, 0.83128349, 0.83129946,
       0.82871515, 0.82769381, 0.82977671, 0.83036305, 0.8280578 ,
       0.82397384, 0.82088493, 0.82114766, 0.8218365 ]), 0.8318605526680098), 'concordance_index_ipcw

# Neural Network - DeepHit

In [36]:
#import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 

import torch # For building the networks 
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv

In [37]:
np.random.seed(1234)
_ = torch.manual_seed(123)

In [38]:
cols_standardize = train[features].columns.tolist()
cols_leave = []

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]

x_mapper = DataFrameMapper(standardize + leave)

In [39]:
x_train = x_mapper.fit_transform(train).astype('float32')
x_test = x_mapper.transform(test).astype('float32')
x_val = x_mapper.transform(val).astype('float32')

In [40]:
num_durations = 70
labtrans = DeepHitSingle.label_transform(num_durations)
get_target = lambda df: (df['time_to_event'].values, df['status'].values)
y_train = labtrans.fit_transform(*get_target(train))
y_val = labtrans.transform(*get_target(val))

train_NN = (x_train, y_train)
val = (x_val, y_val)

In [41]:
x_train.shape

(12613, 51)

In [42]:
in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = labtrans.out_features
batch_norm = True
dropout = 0.1

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)

In [43]:
model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

In [44]:
batch_size = 256
lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=2)

In [45]:
model.optimizer.set_lr(lr_finder.get_best_lr())

In [46]:
epochs = 100
callbacks = [tt.callbacks.EarlyStopping()]
log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

0:	[0s / 0s],		train_loss: 0.3070,	val_loss: 0.1758
1:	[0s / 0s],		train_loss: 0.1706,	val_loss: 0.1654
2:	[0s / 1s],		train_loss: 0.1639,	val_loss: 0.1640
3:	[0s / 1s],		train_loss: 0.1618,	val_loss: 0.1617
4:	[0s / 1s],		train_loss: 0.1563,	val_loss: 0.1594
5:	[0s / 2s],		train_loss: 0.1519,	val_loss: 0.1639
6:	[0s / 2s],		train_loss: 0.1538,	val_loss: 0.1586
7:	[0s / 3s],		train_loss: 0.1501,	val_loss: 0.1545
8:	[0s / 3s],		train_loss: 0.1475,	val_loss: 0.1579
9:	[0s / 3s],		train_loss: 0.1497,	val_loss: 0.1537
10:	[0s / 4s],		train_loss: 0.1456,	val_loss: 0.1509
11:	[0s / 4s],		train_loss: 0.1457,	val_loss: 0.1498
12:	[0s / 5s],		train_loss: 0.1445,	val_loss: 0.1497
13:	[0s / 5s],		train_loss: 0.1461,	val_loss: 0.1506
14:	[0s / 6s],		train_loss: 0.1399,	val_loss: 0.1465
15:	[0s / 6s],		train_loss: 0.1383,	val_loss: 0.1473
16:	[0s / 6s],		train_loss: 0.1491,	val_loss: 0.1522
17:	[0s / 7s],		train_loss: 0.1382,	val_loss: 0.1432
18:	[0s / 7s],		train_loss: 0.1423,	val_loss: 0.1460
19:

In [47]:
surv = model.interpolate(10).predict_surv_df(x_test)

In [48]:
durations_test, events_test = get_target(test)

In [49]:
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td('antolini')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


0.9586742683134227

# Neural Network - DeepSurv

In [31]:
#import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 

import torch # For building the networks 
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import CoxCC
from pycox.evaluation import EvalSurv

In [32]:
np.random.seed(123)
_ = torch.manual_seed(1234)

In [33]:
in_features = x_train.shape[1]
num_nodes = [32, 32]
out_features = 1
batch_norm = True
dropout = 0.1
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, output_bias=output_bias)

In [34]:
model = CoxCC(net, tt.optim.Adam)

In [35]:

batch_size = 256
lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance=2)

In [36]:
model.optimizer.set_lr(lr_finder.get_best_lr())

In [37]:
epochs = 256
callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

log = model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)

0:	[0s / 0s],		train_loss: 0.4185,	val_loss: 0.4784
1:	[0s / 0s],		train_loss: 0.2910,	val_loss: 0.3235
2:	[0s / 0s],		train_loss: 0.2568,	val_loss: 0.3707
3:	[0s / 0s],		train_loss: 0.2796,	val_loss: 0.3080
4:	[0s / 0s],		train_loss: 0.2367,	val_loss: 0.3551
5:	[0s / 0s],		train_loss: 0.2382,	val_loss: 0.3246
6:	[0s / 0s],		train_loss: 0.2383,	val_loss: 0.3050
7:	[0s / 0s],		train_loss: 0.2239,	val_loss: 0.2758
8:	[0s / 0s],		train_loss: 0.2144,	val_loss: 0.2528
9:	[0s / 0s],		train_loss: 0.2275,	val_loss: 0.2270
10:	[0s / 0s],		train_loss: 0.2173,	val_loss: 0.3012
11:	[0s / 0s],		train_loss: 0.2025,	val_loss: 0.2415
12:	[0s / 0s],		train_loss: 0.1986,	val_loss: 0.2597
13:	[0s / 0s],		train_loss: 0.2028,	val_loss: 0.3292
14:	[0s / 0s],		train_loss: 0.2092,	val_loss: 0.3108
15:	[0s / 0s],		train_loss: 0.2153,	val_loss: 0.2118
16:	[0s / 0s],		train_loss: 0.2106,	val_loss: 0.2498
17:	[0s / 0s],		train_loss: 0.2023,	val_loss: 0.2895
18:	[0s / 0s],		train_loss: 0.1871,	val_loss: 0.2389
19:

In [38]:
_ = model.compute_baseline_hazards()

surv = model.predict_surv_df(x_test)

In [39]:
durations_test, events_test = get_target(test)

In [40]:
ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
ev.concordance_td()

0.9247491378316661

# NGBoost

In [87]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from ngboost import NGBSurvival
from ngboost.distns import LogNormal

In [88]:
features = ['CHANNEL.B', 'CHANNEL.C', 'CHANNEL.R', 'NUM_BO.1', 'NUM_BO.2',
       'NUM_BO.3', 'NUM_BO.4', 'FIRST_FLAG.N', 'FIRST_FLAG.Y', 'PURPOSE.C',
       'PURPOSE.P', 'PURPOSE.R', 'PROP.CO', 'PROP.CP', 'PROP.MH', 'PROP.PU',
       'PROP.SF', 'OCC_STAT.I', 'OCC_STAT.P', 'OCC_STAT.S',
       'HIGH_BALANCE_LOAN_INDICATOR.N', 'HIGH_BALANCE_LOAN_INDICATOR.Y',
       'ORIG_RATE', 'CURR_RATE', 'ORIG_UPB', 'ORIG_TERM', 'OLTV', 'OCLTV',
       'DTI', 'CSCORE_B', 'CSCORE_C', 'MI_PCT', 'NO_UNITS.1', 'NO_UNITS.2',
       'NO_UNITS.3', 'NO_UNITS.4', 'MI_TYPE.BPMI', 'MI_TYPE.LPMI',
       'MI_TYPE.None', 'max_deliq_6m',
       'max_deliq_12m', 'max_deliq_6_12', 'if_deliq', 'deliq_stat_6m',
       'deliq_stat_12m', 'deliq_stat_avg_3_12', 'deliq_stat_avg_6_12',
       'ACT_PERIOD', 'CURRENT_UPB', 'LOAN_AGE',
       'ADJ_REM_MONTHS']
target = ['time_to_event']
censoring = ["status"]

In [93]:
import random
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")

# Define your parameter grid here
b1 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=2)
b2 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=3)
b3 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=4)
b4 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=6)
b5 = DecisionTreeRegressor(criterion='friedman_mse', max_depth=8)

param_grid = {
    'learning_rate': [0.01, 0.1, .05, 1],
    'minibatch_frac': [1.0, 0.5],
    'n_estimators':  [75, 100, 200, 300, 400],
    'Base':[b1, b2, b3, b4, b5]
}

X_NGB = train[features].reset_index()
y_NGB = np.array(train.time_to_event.values.tolist())
e_NGB = np.array(train.status.values.tolist())

# Define your number of folds for cross-validation
n_folds = 3

# Define your number of random hyperparameter combinations to try
n_combinations = 50

# Define an empty list to store the results of each hyperparameter combination
results = []

# Loop through each hyperparameter combination
for i in range(n_combinations):
    
    print("done with combination nr.", i )
    
    # Sample a random combination of hyperparameters from the parameter grid
    params = {
        'learning_rate': random.choice(param_grid['learning_rate']),
        'minibatch_frac': random.choice(param_grid['minibatch_frac']),
        'n_estimators': random.choice(param_grid['n_estimators']),
        'Base': random.choice(param_grid['Base'])
    }
    
    # Define an empty list to store the results of each fold
    fold_results = []
    
    # Split the data into folds for cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True)
    for train_index, test_index in kf.split(X_NGB):
        
        # Split the data into training and testing sets for this fold
        X_train_NGB, X_test_NGB = X_NGB.iloc[train_index], X_NGB.iloc[test_index]
        y_train_NGB, y_test_NGB = y_NGB[train_index], y_NGB[test_index]
        e_train_NGB, e_test_NGB = e_NGB[train_index], e_NGB[test_index]
        
        # Train your model with the current hyperparameters
        model = NGBSurvival(Dist=LogNormal, **params)
        model.fit(X_train_NGB, y_train_NGB, e_train_NGB)
        
        
        train_CI = train.reset_index().iloc[train_index]
        
        y_train_CI = np.array(
            list(zip(train_CI['status'], train_CI['time_to_event'])),
            dtype=[('status', '?'), ('time_to_event', 'double')]
        )
        
        
        test_CI = train.reset_index().iloc[test_index]
            
        y_test_CI = np.array(
            list(zip(test_CI['status'], test_CI['time_to_event'])),
            dtype=[('status', '?'), ('time_to_event', 'double')]
        )
        
        # Make predictions on the test set and calculate the accuracy
        y_pred = model.pred_dist(X_test_NGB)
        
        try:
            ci = concordance_index_ipcw(y_train_CI, y_test_CI, -y_pred.mean())[0]
        except:
            ci = 0
            
        
        # Append the accuracy to the list of results for this fold
        fold_results.append(ci)
        

    
    # Calculate the average accuracy across all folds for this hyperparameter combination
    avg_ci = sum(fold_results) / n_folds
    
    # Append the hyperparameter combination and its average accuracy to the list of results
    results.append((params, avg_ci))
    
# Print the hyperparameter combination with the highest average accuracy
best_params, best_ci = max(results, key=lambda x: x[1])
print(f"Best hyperparameters: {best_params}")
print(f"Highest concordance index: {best_ci}")

done with combination nr. 0
[iter 0] loss=1.4091 val_loss=0.0000 scale=2.0000 norm=2.2946
[iter 100] loss=0.6578 val_loss=0.0000 scale=2.0000 norm=0.9093
[iter 0] loss=1.4282 val_loss=0.0000 scale=2.0000 norm=2.3314
[iter 100] loss=0.6349 val_loss=0.0000 scale=2.0000 norm=0.9164
[iter 0] loss=1.4357 val_loss=0.0000 scale=2.0000 norm=2.3345
[iter 100] loss=0.6635 val_loss=0.0000 scale=2.0000 norm=0.9216
done with combination nr. 1
[iter 0] loss=1.4177 val_loss=0.0000 scale=4.0000 norm=4.6471
[iter 0] loss=1.4233 val_loss=0.0000 scale=8.0000 norm=9.3027
[iter 0] loss=1.4265 val_loss=0.0000 scale=4.0000 norm=4.6485
done with combination nr. 2
[iter 0] loss=1.4114 val_loss=0.0000 scale=8.0000 norm=9.1934
[iter 100] loss=0.5840 val_loss=0.0000 scale=2.0000 norm=0.6750
[iter 200] loss=0.5965 val_loss=0.0000 scale=2.0000 norm=0.6298
[iter 0] loss=1.4243 val_loss=0.0000 scale=4.0000 norm=4.6637
[iter 100] loss=0.6124 val_loss=0.0000 scale=2.0000 norm=0.7125
[iter 200] loss=0.6085 val_loss=0.00

In [94]:
def NGB_survival(train, test, features, target, censoring, params):
    clf = NGBSurvival(Dist=LogNormal, random_state = 42, **params)
    Y_train = np.array(train.time_to_event.values.tolist())
    E_train = np.array(train.status.values.tolist())
    clf.fit(train[features], Y_train, E_train)
    preds = clf.pred_dist(test[features])
    return preds, clf

def NGB_survival_result_wrapper(train, test, features, target, censoring, y_train, y_test, times, verbose, params):
    model_output, model = NGB_survival(
        train, test, features, target, censoring, params
    )
    
    results = calc_metrics(y_train, y_test, -model_output.mean(), times)
    return results, model, -model_output.mean()

In [96]:
results_NGB = NGB_survival_result_wrapper(
    train,
    test,
    features,
    target,
    censoring,
    y_train,
    y_test,
    times_data,
    False,
    best_params
)

print(results_NGB)

[iter 0] loss=1.4263 val_loss=0.0000 scale=2.0000 norm=2.3360
[iter 100] loss=0.4083 val_loss=0.0000 scale=2.0000 norm=0.3429
({'cumulative_dynamic_auc': (array([0.98162915, 0.97214889, 0.97684449, 0.97351828, 0.97371318,
       0.97642671, 0.97758091, 0.97980852, 0.97996872, 0.98058521,
       0.98152646, 0.98237221, 0.98223945, 0.98253665, 0.98268761,
       0.98335247, 0.98410531, 0.98319479, 0.98355727, 0.98046528,
       0.97803362, 0.97890951, 0.97917655, 0.98084095, 0.98215743,
       0.98329194, 0.9842116 , 0.9848041 , 0.9840475 , 0.98442952,
       0.98451682, 0.98494622, 0.98554814, 0.98480003, 0.98387289,
       0.98456765, 0.98487587, 0.98538845, 0.98570511, 0.98614576,
       0.98649364, 0.98396468, 0.9827605 , 0.98386029, 0.98375093,
       0.98323208, 0.98372718, 0.98341762, 0.98353179, 0.96944143,
       0.96171707, 0.95976517, 0.96066848, 0.95903888, 0.96020078,
       0.95963358, 0.96048108, 0.96132444, 0.9611078 , 0.95967165,
       0.96067283, 0.95880133, 0.9557723 

In [92]:
model_NGB = results_NGB[1]
NGB_results = results_NGB[2]

In [68]:
model_NGB.save('NGB_model.h5')
np.save('x_test.npy', test[features])
np.save('y_test.npy', test[['time_to_event', 'status']])
np.save('preds.npy', NGB_results)

AttributeError: 'NGBSurvival' object has no attribute 'save'

[CV 2/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.916 total time=   5.7s
[CV 3/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.920 total time=   6.5s
[CV 1/3] END max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=10;, score=0.918 total time=   6.7s
